In [1]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb import OutputHandler
from tblite.interface import Calculator


dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

# numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
# positions = torch.tensor([
#                 [-3.81469488143921, +0.09993441402912, 0.00000000000000],
#                 [+3.81469488143921, -0.09993441402912, 0.00000000000000],
#                 [-2.66030049324036, -2.15898251533508, 0.00000000000000],
#                 [+2.66030049324036, +2.15898251533508, 0.00000000000000],
#                 [-0.73178529739380, -2.28237795829773, 0.00000000000000],
#                 [-5.89039325714111, -0.02589114569128, 0.00000000000000],
#                 [-3.71254944801331, -3.73605775833130, 0.00000000000000],
#                 [+3.71254944801331, +3.73605775833130, 0.00000000000000],
#                 [+0.73178529739380, +2.28237795829773, 0.00000000000000],
#                 [+5.89039325714111, +0.02589114569128, 0.00000000000000],
#                 [-2.74426102638245, +2.16115570068359, 0.00000000000000],
#                 [+2.74426102638245, -2.16115570068359, 0.00000000000000],
#                 ], **dd) # ** to use dd as kwargs

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
cache_config = ConfigCache(enabled=True, density=True, fock=True, overlap=True, hcore=True)
cache_config = ConfigCache(enabled=True, density=True, fock=True, overlap=True, hcore=True, mo_energies=True, coefficients=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config
OutputHandler.verbosity = 0

# instantiate a tblite calculator
calc_tblite = Calculator(
    method = "GFN1-xTB",
    numbers = numbers.numpy(),
    positions = positions.numpy(),
    uhf=None
)

calc_tblite.set("save-integrals", 1)
calc_tblite.set("verbosity", 0)
res = calc_tblite.singlepoint()

# Check if output dxtb and tblite are equal

In [2]:
from qcm_ml.features.xtb import _get_permutation_map_closedshell_convention, apply_perm_map
from dxtb.integrals.wrappers import overlap

perm_map_dxtb = _get_permutation_map_closedshell_convention(numbers.numpy(), "dxtb")
perm_map_tblite = _get_permutation_map_closedshell_convention(numbers.numpy(), "tblite")

p_dxtb = apply_perm_map(calc.get_density(pos), perm_map_dxtb, positions.numpy())
p_tblite = apply_perm_map(res["density-matrix"], perm_map_tblite, positions.numpy())

# E_dxtb, C_dxtb, S_dxtb = calc.get_mo_energies(pos), calc.get_coefficients(pos), overlap(numbers, pos, dxtb.GFN1_XTB)
# F_dxtb = apply_perm_map(torch.einsum('ij,jk,kl,lm->im', S_dxtb, C_dxtb, torch.diag(E_dxtb), torch.inverse(C_dxtb)), perm_map_dxtb, positions.numpy())
F_dxtb = apply_perm_map(calc.cache["fock"], perm_map_dxtb, positions.numpy())

E_tblite, C_tblite, S_tblite = res["orbital-energies"], res["orbital-coefficients"], res["overlap-matrix"]
F_tblite = apply_perm_map(np.einsum('ij,jk,kl,lm->im', S_tblite, C_tblite, np.diag(E_tblite), np.linalg.inv(C_tblite)), perm_map_tblite, positions.numpy())

assert torch.allclose(p_dxtb, torch.tensor(p_tblite, **dd), atol=1e-3)
assert torch.allclose(F_dxtb, torch.tensor(F_tblite, **dd), atol=1e-4)

# Check if 5 point stencil and manual jacobian are equal

In [4]:
from qcm_ml.features.xtb import get_jacobian, get_2body_grads


P_jac_dxtb = get_jacobian(p_dxtb, pos)
F_jac_dxtb = get_jacobian(F_dxtb, pos)

two_body_grads = get_2body_grads(numbers.numpy(), positions.numpy())
P_jac_tblite = two_body_grads[1]
F_jac_tblite = two_body_grads[0]

In [5]:
print(f"Max diff in P_jac: {torch.max(torch.abs(P_jac_dxtb - torch.tensor(P_jac_tblite, **dd))):.2e}"
      f" with max value in P_jac: {torch.max(torch.abs(P_jac_dxtb)): .2e}")
print(f"Max diff in F_jac: {torch.max(torch.abs(F_jac_dxtb - torch.tensor(F_jac_tblite, **dd))):.2e}"
        f" with max value in F_jac: {torch.max(torch.abs(F_jac_dxtb)): .2e}")

print(f"P_jac_dxtb slice: {P_jac_dxtb[:1, :1, :2, :]}")
print(f"F_jac_dxtb slice: {F_jac_dxtb[:1, :1, :2, :]}")

Max diff in P_jac: 3.13e-02 with max value in P_jac:  1.63e-01
Max diff in F_jac: 7.62e-03 with max value in F_jac:  1.46e-01
P_jac_dxtb slice: tensor([[[[ 4.8669e-17, -1.4612e-17,  8.9727e-02],
          [-4.8669e-17,  1.4612e-17, -8.9727e-02]]]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)
F_jac_dxtb slice: tensor([[[[-2.1259e-17,  6.0837e-18,  1.2514e-03],
          [ 2.1259e-17, -6.0837e-18, -1.2514e-03]]]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)


# Auto jac

In [5]:
def get_p(pos):
    return apply_perm_map(calc.get_density(pos), perm_map_dxtb, positions.numpy())
def get_f(pos):
    E_dxtb, C_dxtb, S_dxtb = calc.get_mo_energies(pos), calc.get_coefficients(pos), overlap(numbers, pos, dxtb.GFN1_XTB)
    F_dxtb = apply_perm_map(torch.einsum('ij,jk,kl,lm->im', S_dxtb, C_dxtb, torch.diag(E_dxtb), torch.inverse(C_dxtb)), perm_map_dxtb, positions.numpy())
    return F_dxtb

P_jac_auto = torch.autograd.functional.jacobian(get_p, pos, strict=True)
F_jac_auto = torch.autograd.functional.jacobian(get_f, pos, strict=True)

RuntimeError: Output 0 of the user-provided function is independent of input 0. This is not allowed in strict mode.

In [41]:
print(f"Max diff in P_jac: {torch.max(torch.abs(P_jac_auto - P_jac_dxtb)): .2e}"
      f" with max value in P_jac: {torch.max(torch.abs(P_jac_auto)): .2e}")
print(f"Max diff in F_jac: {torch.max(torch.abs(F_jac_auto - F_jac_dxtb)): .2e}"
        f" with max value in F_jac: {torch.max(torch.abs(F_jac_auto)): .2e}")

print(f"P_jac_auto slice: {P_jac_auto[:1, :1, :2, :]}")
print(f"F_jac_auto slice: {F_jac_auto[:1, :1, :2, :]}")

Max diff in P_jac:  1.63e-01 with max value in P_jac:  0.00e+00
Max diff in F_jac:  1.08e-01 with max value in F_jac:  7.87e-02
P_jac_auto slice: tensor([[[[0., 0., 0.],
          [0., 0., 0.]]]], dtype=torch.float64)
F_jac_auto slice: tensor([[[[ 0.0000,  0.0000, -0.0233],
          [ 0.0000,  0.0000,  0.0233]]]], dtype=torch.float64)
